In [1]:
# Initial Imports
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
import time
from joblib import dump,load # Save Models
from numpy import random
import os
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt

# SDK Imports
import tweepy
import shrimpy
from newsapi import NewsApiClient
import cryptocompare

# NLP Models
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
#from ibm_watson import ToneAnalyzerV3
#from ibm_cloud_sdk.authenticators import IAMAuthenticator
# SKLearn Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler, MinMaxScalar
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
# Tensor Flow Models
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense, Dropout

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# HV Plot
#import hvplot.pandas
#import panel as pn
#from plotly.express as px

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\grant\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Set environment variables from the .env file
env_path = Path("/Users/GrantDePalma")/'.env'
load_dotenv(env_path)


# Extract API Keys from environment variables

shrimpy_public_key = os.getenv("SHRIMPY_PUBLIC_KEY")
shrimpy_private_key = os.getenv("SHRIMPY_PRIVATE_KEY")
TWITTER_API_KEY = os.getenv("TWITTER_API_KEY")
TWITTER_API_SECRET_KEY = os.getenv("TWITTER_API_SECRET_KEY")
TWITTER_ACCESS_TOKEN = os.getenv("TWITTER_ACCESS_TOKEN")
TWITTER_ACCESS_TOKEN_SECRET = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")
KRAKEN_API_KEY = os.getenv("KRAKEN_API_KEY")
KRAKEN_PRIVATE_KEY = os.getenv("KRAKEN_PRIVATE_KEY")
NEWS_API_KEY = os.getenv("NEWS_API_KEY")
#BITTREX_PRIVATE_KEY = 
#BITTREX_PUBLIC_KEY = 





In [3]:
# Twitter API client
auth_twitter = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_SECRET_KEY)
auth_twitter.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
twitter_client = tweepy.API(auth_twitter)

# Shrimpy API Client
shrimpy_client = shrimpy.ShrimpyApiClient(shrimpy_public_key, shrimpy_private_key)

# News API Client
newsapi = NewsApiClient(api_key=NEWS_API_KEY)


## Data Pre-Processing

In [4]:
# Kraken Assets
kraken_assets = shrimpy_client.get_exchange_assets('kraken')
kraken_assets_df = pd.DataFrame(columns=['id','name', 'symbol', 'trading_symbol'])
for key, value in enumerate(kraken_assets):
    kraken_assets_df.loc[key] = [value['id'], value['name'], value['symbol'], value['tradingSymbol']]
kraken_assets_df.head()

,id,name,symbol,trading_symbol
0,429,Bitcoin Cash,BCH,BCH
1,38,Bitcoin,BTC,XBT
2,108,Dash,DASH,DASH
3,152,EOS,EOS,EOS
4,180,Gnosis,GNO,GNO


In [5]:
kraken_assets_df

,id,name,symbol,trading_symbol
0,429,Bitcoin Cash,BCH,BCH
1,38,Bitcoin,BTC,XBT
2,108,Dash,DASH,DASH
3,152,EOS,EOS,EOS
4,180,Gnosis,GNO,GNO
5,154,Ethereum Classic,ETC,ETC
6,153,Ethereum,ETH,ETH
7,229,Litecoin,LTC,LTC
8,244,Melon,MLN,MLN
9,26,Augur,REP,REP


In [6]:
# Kraken Pairs
kraken_usd_pairs = ['XBT_USD', 'BCH_USD', 'DASH_USD', 'XDG_USD','EOS_USD', 'ETH_USD', 'LTC_USD', 'XLM_USD', 'XMR_USD', 'XRP_USD', 'ZEC_USD', 'ADA_USD', 'XTZ_USD']
kraken_xbt_pairs = ['BCH_XBT', 'DASH_XBT', 'XDG_XBT', 'EOS_XBT', 'ETH_XBT', 'LTC_XBT', 'XLM_XBT', 'XMR_XBT', 'XRP_XBT', 'ZEC_XBT', 'ADA_XBT', 'XTZ_XBT']
kraken_watchlist = ['XBT', 'BCH', 'DASH', 'XDG', 'EOS', 'ETH', 'LTC', 'XLM', 'XMR', 'XRP', 'ZEC', 'ADA', 'XTZ']

In [11]:


def fetch_data():
    # Retrieve available assets on kraken exchange from shrimpy api client
    kraken_assets = shrimpy_client.get_exchange_assets('kraken')
    kraken_assets_df = pd.DataFrame(columns=['id','name', 'symbol', 'trading_symbol'])
    for key, value in enumerate(kraken_assets):
        kraken_assets_df.loc[key] = [value['id'], value['name'], value['symbol'], value['tradingSymbol']]
    
    # Retrieve available pairs traded on kraken exchange from shrimpy api client 
    def calc_kraken_pairs_dataframe():
        kraken_pairs = shrimpy_client.get_trading_pairs('kraken')
        kraken_pairs_df = pd.DataFrame(columns=['base','quote'])
        for key, value in enumerate(kraken_pairs):
            kraken_pairs_df.loc[key] = [value['baseTradingSymbol'],value['quoteTradingSymbol']]    
        return kraken_pairs_df
    
    def calc_features():
        
        return features_df

    
    return master_ml_dataframe


def universe_selection(): # enumeration
    # Filters = all kraken, usd quote, btc quote, btcusd, watchlist_btc, watchlist_usd
    
    return

def feature_selection():
    # Nueral Networks-Regression models 
    def make_pipeline():
       #pipe =  Pipeline(
       #    columns={'M_Cap': ,
       #             'price_velocity': ,
       #             'price_acceleration': ,
       #             'sentiment_velocity':,
       #             'sentiment_acceleration':,
       #             'ema_cross':,
       #             'vol_cross':,
       #             'RSI':,
       #             'PC 1':,
       #             'PC 2':,
       #             'PC 3':,
       #            },
       #screen = universe_selection()
       #)
        return
    return


def on_tick():
    return

def target_portfolio():
    return

def dynamic_rebalance():
    wts = []
    return
    

In [12]:
def calc_kraken_pairs_dataframe():
    kraken_pairs = shrimpy_client.get_trading_pairs('kraken')
    kraken_pairs_df = pd.DataFrame(columns=['base','quote'])
    for key, value in enumerate(kraken_pairs):
        kraken_pairs_df.loc[key] = [value['baseTradingSymbol'],value['quoteTradingSymbol']]    
    return kraken_pairs_df

In [13]:
kraken_pairs_df = calc_kraken_pairs_dataframe()

In [ ]:
kraken_usd_pairs_df = kraken_pairs_df[(kraken_pairs_df['quote'] == 'USD') & (kraken_pairs_df['base'].isin(kraken_watchlist))]
kraken_xbt_pairs_df = kraken_pairs_df[(kraken_pairs_df['quote'] == 'XBT') & (kraken_pairs_df['base'].isin(kraken_watchlist))]

In [ ]:
kraken_usd_pairs_df.head(12)

In [ ]:
kraken_xbt_pairs_df

In [14]:
# ## Prices
def get_prices(kraken_pairs_df, interval, starttime):
    master_prices_df = pd.DataFrame()
    time = []
    prices = []
    for index, row in kraken_pairs_df.iterrows():
        candles = shrimpy_client.get_candles('kraken', row['base'], row['quote'], interval, starttime)
        for key, value in enumerate(candles):
            #prices_df.loc[key] = [pd.to_datetime(value['time']), value['close']]
            #time.append(pd.to_datetime(value['time']))
            time.append(value['time'])
            prices.append(value['close'])
            prices_df = pd.DataFrame(list(zip(time, prices)), columns = ['time', row['base']+ "_" + row['quote']])
        if master_prices_df.empty:
            master_prices_df = prices_df
        else:
            master_prices_df = pd.merge(master_prices_df, prices_df, on='time', how='left')        
    return master_prices_df


In [19]:
df_prices_kraken_1st10_1m = get_prices(kraken_pairs_df.head(), '1m', '2016-01-01')

In [20]:
df_prices_kraken_1st10_1m

,time,ADA_ETH,ADA_EUR,ADA_USD,ADA_XBT,ALGO_ETH
0,2020-08-07T10:21:20.000Z,0.0003649000000000,0.0003649000000000,0.0003649000000000,0.0003649000000000,0.0003649000000000
1,2020-08-07T10:21:20.000Z,0.0003649000000000,0.0003649000000000,0.0003649000000000,0.0003649000000000,0.1227150000000000
2,2020-08-07T10:21:20.000Z,0.0003649000000000,0.0003649000000000,0.0003649000000000,0.1227150000000000,0.0003649000000000
3,2020-08-07T10:21:20.000Z,0.0003649000000000,0.0003649000000000,0.0003649000000000,0.1227150000000000,0.1227150000000000
4,2020-08-07T10:21:20.000Z,0.0003649000000000,0.0003649000000000,0.1227150000000000,0.0003649000000000,0.0003649000000000
...,...,...,...,...,...,...
1548,2020-08-08T02:51:20.000Z,0.0003655000000000,0.0003655000000000,0.0003655000000000,0.0003655000000000,0.0003655000000000
1549,2020-08-08T02:51:21.000Z,0.0003652000000000,0.0003652000000000,0.0003652000000000,0.0003652000000000,0.0003652000000000
1550,2020-08-08T02:51:36.000Z,0.0003657000000000,0.0003657000000000,0.0003657000000000,0.0003657000000000,0.0003657000000000
1551,2020-08-08T02:52:22.000Z,0.0003656000000000,0.0003656000000000,0.0003656000000000,0.0003656000000000,0.0003656000000000


In [17]:
# set index to time
df_prices_usd_1m = df_prices_kraken_1st10_1m.set_index('time')
df_prices_usd_1m.head()

,ZEC_USD,EUR_USD,GBP_USD,USD_CAD,USD_JPY
time,,,,,
2020-08-07T10:28:01.000Z,96.8100000000000000,96.8100000000000000,96.8100000000000000,96.8100000000000000,96.8100000000000000
2020-08-07T10:31:10.000Z,96.6800000000000000,96.6800000000000000,96.6800000000000000,96.6800000000000000,96.6800000000000000
2020-08-07T10:31:46.000Z,96.6800000000000000,96.6800000000000000,96.6800000000000000,96.6800000000000000,96.6800000000000000
2020-08-07T10:47:07.000Z,96.1900000000000000,96.1900000000000000,96.1900000000000000,96.1900000000000000,96.1900000000000000
2020-08-07T10:51:05.000Z,96.0000000000000000,96.0000000000000000,96.0000000000000000,96.0000000000000000,96.0000000000000000


In [18]:
# convert prices to floats
df_prices_usd_1m = df_prices_usd_1m.apply(pd.to_numeric)
df_prices_usd_1m.head()

,ZEC_USD,EUR_USD,GBP_USD,USD_CAD,USD_JPY
time,,,,,
2020-08-07T10:28:01.000Z,96.81,96.81,96.81,96.81,96.81
2020-08-07T10:31:10.000Z,96.68,96.68,96.68,96.68,96.68
2020-08-07T10:31:46.000Z,96.68,96.68,96.68,96.68,96.68
2020-08-07T10:47:07.000Z,96.19,96.19,96.19,96.19,96.19
2020-08-07T10:51:05.000Z,96.00,96.00,96.00,96.00,96.00


In [ ]:
def calc

In [ ]:
df_candles_t2 = df_candles_t.apply(pd.to_numeric)
df_candles_t2.head()

In [ ]:
# Calculate Cross Sectional Factors
df_price_velocity = df_candles_t2.pct_change()
df_price_acceleration = df_price_velocity.pct_change()
df_price_acceleration.head()

In [ ]:
# Technical Factors
short_window = 9
long_window = 21

df_ema_fast = df_price_velocity.ewm(halflife=short_window).mean()
df_ema_fast.head()

In [ ]:
df_ema_slow = df_price_velocity.ewm(halflife=long_window).mean()
df_ema_slow.head(75)

In [ ]:
df_ema_slow.head()

In [ ]:
df_ema_fast.head()

In [ ]:
df_ema_crossover_long = np.where(df_ema_fast > df_ema_slow, 1.0, 0.0)

In [ ]:
df_ema_crossover_long = pd.DataFrame(df_ema_crossover_long)
df_ema_crossover_long.head()

In [ ]:

df_candles_stack = df_candles_t2.stack().to_frame(name='price_velocity')
df_candles_stack.head(20)

In [ ]:
shrimpy.

In [ ]:
# Shrimpy Data Prep

def fetch_shrimpy_data():
    
    shrimpy_candles # 1m, 5m, 15m, 30m, 60m, 4h, 1d candles for universe of exchange Rates
    shrimpy_exchanges # kraken, coinbase pro, bittrex
    shrimpy_rates  # kraken, coinbase pro, bittrex available crypto exchange rates #
    
    
    

In [ ]:
# CryptoCompare Data Prep


In [ ]:
# NewsAPI Data Prep


In [ ]:
# Twitter Data Prep

tweets = api_twitter.search('$SPY', count=200)
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
display(data.head(10))

#print(tweets[0].created_at)

# Factor Research Notebook
---

In [ ]:
#Strategy Parameters (Global Variables)
exchanges = ['kraken', 'bittrex', 'coinbasepro']
ema_short_period = 9
ema_long_period = 21
RSI_period = 14
RSI_threshold_top = 70
RSI_threshold_bottom = 30
leverage = 0

In [ ]:
def select_universe():
    return

def calc_factors():
    # Quantitative - Momentum, Mean Reversion
    def price_velocity():
        return price_velocity
    def price_acceleration():
        return price_acceleration
    
# 1hr twitter signal, asset names, nltk intensity - IBM(tone)
    def sentiment_velocity():
        return sentiment_velocity
    def sentiment_acceleration():
        return sentiment_acceleration


    def ema_cross():
        return ema_cross_signal
    def vol_cross():
        return vol_cross_signal
    def RSI():
        return rsi
    
# wait til after 
    def PC_1():
        return PC_1
    def PC_2():
        return PC_2
    def PC_3():
        return PC_3

def make_pipeline():

    pipe =  Pipeline(
        columns={'M_Cap': ,
                 'price_velocity': ,
                 'price_acceleration': ,
                 'sentiment_velocity':,
                 'sentiment_acceleration':,
                 'ema_cross':,
                 'vol_cross':,
                 'RSI':,
                 'PC 1':,
                 'PC 2':,
                 'PC 3':,
                },
    screen = shrimpy_universe()
    )
    
    
    def onTick():
        return
    
    def rebalance():
        return
    def generate_target_position():
        return
    def unit_sizing():
        return
    
    def kill_function():
        return
    
    



In [ ]:
# Trading Functions
def position_size():

In [ ]:
cryptocompare.get_coin_list()

# ML Trading Models
---

In [ ]:
# Design Signal Features


In [ ]:
# Train Model

In [ ]:
# Trade Model

In [ ]:

dump(random_forest_model, 'random_forest_model.joblib')
dump(gradient_booster_model, 'gradient_booster_model.joblib')
dump()